In [105]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet")

In [22]:
start_date = "2015-01-01"
end_date = "2021-06-01"

In [23]:
time_list_old = DataAPI.get_trading_days(start_date, end_date)
time_list = []
for date in time_list_old:
    time_list.append(str(date.year) + "-"+ str(date.month).zfill(2) + "-"+str(date.day).zfill(2))
time_list

['2015-01-05',
 '2015-01-06',
 '2015-01-07',
 '2015-01-08',
 '2015-01-09',
 '2015-01-12',
 '2015-01-13',
 '2015-01-14',
 '2015-01-15',
 '2015-01-16',
 '2015-01-19',
 '2015-01-20',
 '2015-01-21',
 '2015-01-22',
 '2015-01-23',
 '2015-01-26',
 '2015-01-27',
 '2015-01-28',
 '2015-01-29',
 '2015-01-30',
 '2015-02-02',
 '2015-02-03',
 '2015-02-04',
 '2015-02-05',
 '2015-02-06',
 '2015-02-09',
 '2015-02-10',
 '2015-02-11',
 '2015-02-12',
 '2015-02-13',
 '2015-02-16',
 '2015-02-17',
 '2015-02-25',
 '2015-02-26',
 '2015-02-27',
 '2015-03-02',
 '2015-03-03',
 '2015-03-04',
 '2015-03-05',
 '2015-03-06',
 '2015-03-09',
 '2015-03-10',
 '2015-03-11',
 '2015-03-12',
 '2015-03-13',
 '2015-03-16',
 '2015-03-17',
 '2015-03-18',
 '2015-03-19',
 '2015-03-20',
 '2015-03-23',
 '2015-03-24',
 '2015-03-25',
 '2015-03-26',
 '2015-03-27',
 '2015-03-30',
 '2015-03-31',
 '2015-04-01',
 '2015-04-02',
 '2015-04-03',
 '2015-04-07',
 '2015-04-08',
 '2015-04-09',
 '2015-04-10',
 '2015-04-13',
 '2015-04-14',
 '2015-04-

In [61]:
import multiprocessing as mp
import time
from tqdm import tqdm
pool = mp.Pool(10)
t1 = time.time()
result = {}
def test(i):
    data = DataAPI.get_stock_daily_return(i, ti=0, tp=31, price='vwap', period=["1d","2d","3d","4d","5d","10d"], freq='1min').reset_index()
    data['timestamp'] += pd.to_timedelta(5,"m")
    return data.set_index(["timestamp","ticker"])
for i in tqdm(time_list):
    result[i]  = (pool.apply_async(test, args=(i,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 1560/1560 [00:00<00:00, 162287.67it/s]


并行执行时间： 4


In [62]:
target_df = []
for i in tqdm(result):
    target_df.append(result[i].get())
target_df = pd.concat(target_df,axis=0)
target_df

100%|██████████| 1560/1560 [00:00<00:00, 279465.01it/s]


1d_ret    2d_ret    3d_ret    4d_ret    5d_ret  \
timestamp           ticker                                                     
2015-01-05 09:30:00 000001 -0.003120 -0.031001 -0.036996 -0.067138 -0.070556   
                    000002 -0.001994 -0.019423 -0.041521 -0.070923 -0.097998   
                    000004  0.014671  0.048657  0.065601  0.071498  0.038417   
                    000005  0.000000  0.000000  0.000000  0.000000  0.000000   
                    000006 -0.040209 -0.039435 -0.042880 -0.053565 -0.085893   
...                              ...       ...       ...       ...       ...   
2021-06-01 09:30:00 688699 -0.057854 -0.088197 -0.089355 -0.008459 -0.079010   
                    688777 -0.035939 -0.049675 -0.066356 -0.011289 -0.028392   
                    688788 -0.031378 -0.048310 -0.066576 -0.043054 -0.060163   
                    688819  0.025589  0.011450  0.004278  0.060774  0.056411   
                    688981 -0.011507 -0.023025 -0.038161 -0.011591 -0.013660   

                             10d_ret  trading_status  
timestamp           ticker                            
2015-01-05 09:30:00 000001 -0.104853               0  
                    000002 -0.136266               0  
                    000004  0.018452               0  
                    000005  0.000000               3  
                    000006 -0.131916               0  
...                              ...             ...  
2021-06-01 09:30:00 688699 -0.041999               0  
                    688777 -0.060290               0  
                    688788 -0.065377               0  
                    688819 -0.031494               0  
                    688981 -0.028147               0  

[5281482 rows x 7 columns]

In [97]:
# target_df.drop("trading_status",inplace=True,axis=1)
target_df

1d_ret    2d_ret    3d_ret    4d_ret    5d_ret  \
timestamp           ticker                                                     
2015-01-05 09:30:00 000001 -0.003120 -0.031001 -0.036996 -0.067138 -0.070556   
                    000002 -0.001994 -0.019423 -0.041521 -0.070923 -0.097998   
                    000004  0.014671  0.048657  0.065601  0.071498  0.038417   
                    000005  0.000000  0.000000  0.000000  0.000000  0.000000   
                    000006 -0.040209 -0.039435 -0.042880 -0.053565 -0.085893   
...                              ...       ...       ...       ...       ...   
2021-06-01 09:30:00 688699 -0.057854 -0.088197 -0.089355 -0.008459 -0.079010   
                    688777 -0.035939 -0.049675 -0.066356 -0.011289 -0.028392   
                    688788 -0.031378 -0.048310 -0.066576 -0.043054 -0.060163   
                    688819  0.025589  0.011450  0.004278  0.060774  0.056411   
                    688981 -0.011507 -0.023025 -0.038161 -0.011591 -0.013660   

                             10d_ret  
timestamp           ticker            
2015-01-05 09:30:00 000001 -0.104853  
                    000002 -0.136266  
                    000004  0.018452  
                    000005  0.000000  
                    000006 -0.131916  
...                              ...  
2021-06-01 09:30:00 688699 -0.041999  
                    688777 -0.060290  
                    688788 -0.065377  
                    688819 -0.031494  
                    688981 -0.028147  

[5281482 rows x 6 columns]

In [102]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ssr = StandardScaler()
mmr = MinMaxScaler(feature_range=(-10,10))
columns = group.columns
final_list = []
for name,group in tqdm(target_df.groupby("timestamp")):
    fina_df = group.copy()
    fina_df[[str(i) +"_StandardScaler" for i in group.columns]] = ssr.fit_transform(group)
    fina_df[[str(i) +"_MinMaxScaler_10" for i in group.columns]] = mmr.fit_transform(group)
    fina_df[[str(i) +"_StandardScale_MinMax_10" for i in group.columns]] = mmr.fit_transform(ssr.fit_transform(group))
    final_list.append(fina_df)

100%|██████████| 1560/1560 [00:29<00:00, 53.61it/s]


In [111]:
# target_df = pd.concat(final_list,axis=0)
target_df

1d_ret    2d_ret    3d_ret    4d_ret    5d_ret  \
timestamp           ticker                                                     
2015-01-05 09:30:00 000001 -0.003120 -0.031001 -0.036996 -0.067138 -0.070556   
                    000002 -0.001994 -0.019423 -0.041521 -0.070923 -0.097998   
                    000004  0.014671  0.048657  0.065601  0.071498  0.038417   
                    000005  0.000000  0.000000  0.000000  0.000000  0.000000   
                    000006 -0.040209 -0.039435 -0.042880 -0.053565 -0.085893   
...                              ...       ...       ...       ...       ...   
2021-06-01 09:30:00 688699 -0.057854 -0.088197 -0.089355 -0.008459 -0.079010   
                    688777 -0.035939 -0.049675 -0.066356 -0.011289 -0.028392   
                    688788 -0.031378 -0.048310 -0.066576 -0.043054 -0.060163   
                    688819  0.025589  0.011450  0.004278  0.060774  0.056411   
                    688981 -0.011507 -0.023025 -0.038161 -0.011591 -0.013660   

                             10d_ret  1d_ret_StandardScaler  \
timestamp           ticker                                    
2015-01-05 09:30:00 000001 -0.104853              -0.541929   
                    000002 -0.136266              -0.503045   
                    000004  0.018452               0.072339   
                    000005  0.000000              -0.434206   
                    000006 -0.131916              -1.822556   
...                              ...                    ...   
2021-06-01 09:30:00 688699 -0.041999              -2.225986   
                    688777 -0.060290              -1.456454   
                    688788 -0.065377              -1.296282   
                    688819 -0.031494               0.704046   
                    688981 -0.028147              -0.598535   

                            2d_ret_StandardScaler  3d_ret_StandardScaler  \
timestamp           ticker                                                 
2015-01-05 09:30:00 000001              -1.584587              -1.475431   
                    000002              -1.277469              -1.573327   
                    000004               0.528394               0.743830   
                    000005              -0.762266              -0.675178   
                    000006              -1.808308              -1.602709   
...                                           ...                    ...   
2021-06-01 09:30:00 688699              -2.134933              -1.764211   
                    688777              -1.227989              -1.307274   
                    688788              -1.195837              -1.311633   
                    688819               0.211101               0.096075   
                    688981              -0.600554              -0.747091   

                            4d_ret_StandardScaler  ...  \
timestamp           ticker                         ...   
2015-01-05 09:30:00 000001              -1.691426  ...   
                    000002              -1.758539  ...   
                    000004               0.766915  ...   
                    000005              -0.500910  ...   
                    000006              -1.450746  ...   
...                                           ...  ...   
2021-06-01 09:30:00 688699              -0.221912  ...   
                    688777              -0.269707  ...   
                    688788              -0.806231  ...   
                    688819               0.947432  ...   
                    688981              -0.274807  ...   

                            3d_ret_MinMaxScaler_10  4d_ret_MinMaxScaler_10  \
timestamp           ticker                                                   
2015-01-05 09:30:00 000001               -4.136969               -4.679791   
                    000002               -4.310520               -4.784249   
                    000004               -0.202653               -0.853494   
                    000005         

In [112]:
from AlphaNet.Data import convert_to_standard_daily_data_par
convert_to_standard_daily_data_par(df=target_df,output_name = "Return_Data",output_path="/home/wuwenjun/feature_platform/ti0/wuwenjun")